# 303 - Multiples with Small Digits

## Problem Statement

For a positive integer $n$, define $f(n)$ as the least positive multiple of $n$ that, written in base $10$, uses only digits $\le 2$.

Thus $f(2)=2$, $f(3)=12$, $f(7)=21$, $f(42)=210$, $f(89)=1121222$.

Also, $\sum \limits_{n = 1}^{100} {\dfrac{f(n)}{n}} = 11363107$.

Find $\sum \limits_{n=1}^{10000} {\dfrac{f(n)}{n}}$.

## Solution

First, note that the solution below is far from optimal. 

We recursively generate all the numbers with only digits lower or equal to two. Then, for each candidate, we generate all the divisors below 10000. For each divisor, $n$, that has never been encountered, we add $\frac{f(n)}{n}$ to a running sum storing the results. When generating all the candidates with 15 digits or less, a few values of $n$ are never found (4995, 9990 and 9999). For these, we look for a pattern. Looking at 9, 99 and 999, we observe that the multiple for 9 is 12222 (one 1 followed by four 2), the multiple for 99 is 1122222222 (two 1 followed by eight 2) and the multiple for 999 is 111222222222222 (three 1 followed by twelve 2). Naturally, we guess that the multiple for 9999 is 11112222222222222222 (four 1 followed by sixteen 2). A similar pattern is found for 4995 and 9990 and for both of them the multiple is 10 times the multiple for 999.

As often, we can use numba to speed up computations.

In [5]:
from numba import jit, njit
import numpy as np

def generate_numbers_recursively(current_number, max_digits, results, is_first_digit=True):
    # Append new number to results
    if current_number > 0:
        results.append(current_number)

    # Avoid zero in first digit
    if len(str(current_number)) < max_digits:
        if is_first_digit:
            next_digits = [1, 2]
        else:
            next_digits = [0, 1, 2]

        for digit in next_digits:
            # Build the next number by shifting the current number one decimal place and adding the new digit
            new_number = current_number * 10 + digit
            generate_numbers_recursively(new_number, max_digits, results, False)

def generate_all_numbers(max_digits):
    results = []
    generate_numbers_recursively(0, max_digits, results)
    return results

@njit
def divisors_below_x(n, x):
    divisors = []
    limit = min(x + 1, int(np.sqrt(n)) + 1)
    for i in range(1, limit):
        if n % i == 0:
            if i < x:
                divisors.append(i)
            if n // i < x and n // i != i:
                divisors.append(n // i)
    return divisors

@njit
def compute_sum(candidate_multiples, remaining):
    res = 0
    count = 0
    for num in candidate_multiples:
        divisors = divisors_below_x(num, 10001)
        for d in divisors:
            if remaining[d] == True:
                res += num / d
                count += 1
                remaining[d] = False
        if count == 10000 - 3:
            break
    return res

max_digits = 15
candidate_multiples = generate_all_numbers(max_digits)

remaining = np.array([True] * 10001)

res = compute_sum(np.sort(candidate_multiples), remaining) + 1112222222222220 / 4995 + 1112222222222220 / 9990 + 11112222222222222222 / 9999

int(res)

1111981904675169